In [30]:
import pandas as pd
import numpy as np
# import seaborn as sns
# import matplotlib.pyplot as plt
from scipy.stats import shapiro, normaltest, anderson, spearmanr, kendalltau
import pycountry
import plotly.express as px
import nbformat


# Daten einlesen und Ergebnis überprüfem

In [31]:
df_export = pd.read_csv("data/exports.csv", sep=',', skiprows=4)
df_export = df_export.drop(columns=["Indicator Name", "Indicator Code", "2024", "Unnamed: 69"])
df_export.head()


,Country Name,Country Code,1960,1961,1962,1963,1964,1965,1966,1967,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Aruba,ABW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.184551e+09,2.158564e+09,2.142860e+09,2.243517e+09,2.367134e+09,2.486899e+09,1.315201e+09,2.072292e+09,2.871730e+09,3.223288e+09
1,Africa Eastern and Southern,AFE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.808857e+11,2.167307e+11,1.932475e+11,2.243787e+11,2.571488e+11,2.421184e+11,2.063635e+11,2.824632e+11,3.292074e+11,3.032518e+11
2,Afghanistan,AFG,2.222223e+07,2.444446e+07,26666680.0,6.888892e+07,7.111114e+07,113333340.0,120000000.0,113333340.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2.079571e+09,2.045190e+09,2.664600e+09,2.890868e+09
3,Africa Western and Central,AFW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Angola,AGO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.077040e+10,2.692685e+10,1.483893e+10,2.137317e+10,3.244471e+10,2.891981e+10,1.858315e+10,3.175306e+10,4.634476e+10,3.465662e+10


In [32]:
df_military = pd.read_csv("data/military_expenditure.csv", sep=',', skiprows=4)
df_military = df_military.drop(columns=["Indicator Name", "Indicator Code", "2024", "Unnamed: 69"])
df_military.head()

,Country Name,Country Code,1960,1961,1962,1963,1964,1965,1966,1967,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Aruba,ABW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Africa Eastern and Southern,AFE,NaN,NaN,NaN,3.219951e+08,4.398005e+08,5.239918e+08,5.631079e+08,5.992006e+08,...,1.682935e+10,1.538332e+10,1.365941e+10,1.617268e+10,1.227049e+10,1.169172e+10,1.119335e+10,1.143168e+10,1.212065e+10,1.337060e+10
2,Afghanistan,AFG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.682388e+08,1.995171e+08,1.858783e+08,1.914071e+08,1.980747e+08,2.263062e+08,2.795770e+08,2.782508e+08,NaN,NaN
3,Africa Western and Central,AFW,NaN,6.573052e+07,9.002825e+07,1.005711e+08,1.188867e+08,1.456340e+08,1.507266e+08,2.794051e+08,...,6.451565e+09,5.016333e+09,5.575535e+09,5.510401e+09,6.325122e+09,6.166715e+09,7.186733e+09,9.697806e+09,8.472734e+09,9.463191e+09
4,Angola,AGO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.846249e+09,3.608299e+09,2.764055e+09,3.062873e+09,1.983614e+09,1.470939e+09,9.935944e+08,9.814510e+08,1.622764e+09,1.270158e+09


# Länder filtern
#### nicht Länder werden aussortiert und fälschlich aussortierte wieder einsortiert
#### Länder ohne Daten werden auch rausgefiltert

In [33]:
non_country_keywords = [
    "Africa", "Europe", "America", "Asia", "Oceania", "Arab", "World",
    "High income", "Low income", "Middle income", "Euro area", "OECD", "IDA",
    "IBRD", "G7", "G20", "Least developed", "Landlocked", "Small states", "Sub-Saharan",
    "Heavily indebted", "demographic dividend", "middle income", "Other"
]

allowlist = {
    "Egypt, Arab Rep.", "Saudi Arabia", "South Africa",
    "Syrian Arab Republic", "United Arab Emirates", "Central African Republic"
}

In [34]:
def is_region_or_group(name):
    return (
        any(keyword in str(name) for keyword in non_country_keywords)
        and name not in allowlist
    )

In [35]:
def clean_df(df):
    df = df[~df["Country Name"].apply(is_region_or_group)]
    mask = (df.iloc[:, 3:].fillna(0) != 0).any(axis=1)
    df = df[mask].reset_index(drop=True)
    return df

#### Überprüfen ob nach dem Bereinigen des Datensatzes die übrigen Länder in beiden Datensätzen vorkommen, falls nicht werden sie aussortiert

In [36]:
def filter_for_same_columns(df1, df2):
    set1 = set(df1["Country Name"])
    set2 = set(df2["Country Name"])

    common = set1 & set2

    df1_clean = df1[df1["Country Name"].isin(common)].reset_index(drop=True)
    df2_clean = df2[df2["Country Name"].isin(common)].reset_index(drop=True)
    return df1_clean, df2_clean


In [37]:
df_export = clean_df(df_export)


In [38]:
df_military = clean_df(df_military)


In [39]:
df_export, df_military = filter_for_same_columns(df_export, df_military)

# Spearman Korrelation Export-Militärausgaben

In [40]:
df = df_export.merge(df_military, on="Country Name", suffixes=("_exp", "_mil"))

years = [c for c in df_export.columns if c.isdigit() and c in df_military.columns]

results = []
for _, row in df.iterrows():
    country = row["Country Name"]

    x = pd.to_numeric(row[[f"{y}_exp" for y in years]], errors="coerce").to_numpy()
    y = pd.to_numeric(row[[f"{y}_mil" for y in years]], errors="coerce").to_numpy()
    
    mask = (~np.isnan(x)) & (x != 0) & (~np.isnan(y)) & (y != 0)
    x_valid = x[mask]
    y_valid = y[mask]
    if len(x_valid) < 3:
        continue
    
    # Spearman ρ
    rho, p_s = spearmanr(x_valid, y_valid)
    # Kendall's τ_b
    tau, p_k = kendalltau(x_valid, y_valid)
    
    results.append({
        "Country Name": country,
        "n Jahre": len(x_valid),
        "Spearman ρ": round(rho, 3),
        "p-Spearman": round(p_s, 3),
        "Kendall τ_b": round(tau, 3),
        "p-Kendall": round(p_k, 3)
    })

df_country_corr = pd.DataFrame(results) \
    .sort_values("Spearman ρ", ascending=False) \
    .reset_index(drop=True)

print(df_country_corr)
df_country_corr.to_csv("results/spearman_export_correlations.csv", index=False)


               Country Name  n Jahre  Spearman ρ  p-Spearman  Kendall τ_b  \
0                 Singapore       54       0.992       0.000        0.950   
1                     China       35       0.991       0.000        0.946   
2                 Australia       64       0.991       0.000        0.937   
3                Bangladesh       51       0.991       0.000        0.944   
4                    Mexico       64       0.991       0.000        0.933   
..                      ...      ...         ...         ...          ...   
152                  Guyana       30      -0.160       0.398       -0.076   
153             South Sudan        8      -0.262       0.531       -0.071   
154                 Lao PDR       22      -0.412       0.057       -0.177   
155  Bosnia and Herzegovina       22      -0.486       0.022       -0.333   
156                 Eritrea       11      -0.755       0.007       -0.600   

     p-Kendall  
0        0.000  
1        0.000  
2        0.000  
3      

# Interaktive Weltkarte
#### auf dieser Weltkarte sind für alle Länder welche genügend Daten haben die Korrealtion zwischen ihren Exporten und Militärausgaben aufgezeigt

In [41]:
df_corr = pd.read_csv('results/spearman_export_correlations.csv')

def iso3(name):
    try:
        return pycountry.countries.lookup(name).alpha_3
    except LookupError:
        return None

df_corr['iso_a3'] = df_corr['Country Name'].apply(iso3)

df_map = df_corr.dropna(subset=['iso_a3'])

fig = px.choropleth(
    df_map,
    locations='iso_a3',
    color='Spearman ρ',
    hover_name='Country Name',
    color_continuous_scale='OrRd',
    projection='natural earth',
    title='Interaktive Spearman-ρ Karte: Export vs. Militär'
)

fig.update_geos(showcoastlines=True, showland=True, fitbounds='locations')
fig.update_layout(margin={'r':0,'t':40,'l':0,'b':0})
fig.show()

# Korrelation BIP - Militärausgaben

In [42]:
df_military = pd.read_csv("data/military_expenditure.csv", sep=',', skiprows=4)
df_military = df_military.drop(columns=["Indicator Name", "Indicator Code", "2024", "Unnamed: 69"])
df_military = clean_df(df_military)

In [43]:
df_gdp = pd.read_csv("data/gdp.csv", sep=',', skiprows=4)
df_gdp = df_gdp.drop(columns=["Indicator Name", "Indicator Code", "2024", "Unnamed: 69"])
df_gdp = clean_df(df_gdp)

In [44]:
df_gdp, df_military = filter_for_same_columns(df_gdp, df_military)


In [45]:
df = df_gdp.merge(df_military, on="Country Name", suffixes=("_gdp", "_mil"))

years = [c for c in df_gdp.columns if c.isdigit() and c in df_military.columns]

results = []
for _, row in df.iterrows():
    country = row["Country Name"]

    x = pd.to_numeric(row[[f"{y}_gdp" for y in years]], errors="coerce").to_numpy()
    y = pd.to_numeric(row[[f"{y}_mil" for y in years]], errors="coerce").to_numpy()
    
    mask = (~np.isnan(x)) & (x != 0) & (~np.isnan(y)) & (y != 0)
    x_valid = x[mask]
    y_valid = y[mask]
    
    if len(x_valid) < 3:
        continue
    
    rho, p_s = spearmanr(x_valid, y_valid)
    tau, p_k = kendalltau(x_valid, y_valid)
    
    results.append({
        "Country Name": country,
        "n Jahre": len(x_valid),
        "Spearman ρ": round(rho, 3),
        "p-Spearman": round(p_s, 3),
        "Kendall τ_b": round(tau, 3),
        "p-Kendall": round(p_k, 3)
    })

df_country_corr = pd.DataFrame(results) \
    .sort_values("Spearman ρ", ascending=False) \
    .reset_index(drop=True)

print(df_country_corr)
df_country_corr.to_csv("results/spearman_gdp_correlations.csv", index=False)

               Country Name  n Jahre  Spearman ρ  p-Spearman  Kendall τ_b  \
0                Uzbekistan       10       1.000       0.000        1.000   
1               Korea, Rep.       64       0.997       0.000        0.970   
2                   Estonia       31       0.996       0.000        0.970   
3                     India       64       0.995       0.000        0.959   
4                     China       35       0.995       0.000        0.970   
..                      ...      ...         ...         ...          ...   
160                 Croatia       32       0.339       0.057        0.383   
161            Turkmenistan        6       0.143       0.787        0.200   
162             South Sudan        8      -0.167       0.693       -0.071   
163  Bosnia and Herzegovina       22      -0.203       0.366       -0.143   
164                 Lao PDR       22      -0.309       0.162       -0.108   

     p-Kendall  
0        0.000  
1        0.000  
2        0.000  
3      

In [46]:
df_corr = pd.read_csv('results/spearman_gdp_correlations.csv')

def iso3(name):
    try:
        return pycountry.countries.lookup(name).alpha_3
    except LookupError:
        return None

df_corr['iso_a3'] = df_corr['Country Name'].apply(iso3)

df_map = df_corr.dropna(subset=['iso_a3'])

fig = px.choropleth(
    df_map,
    locations='iso_a3',
    color='Spearman ρ',
    hover_name='Country Name',
    color_continuous_scale='OrRd',
    projection='natural earth',
    title='Interaktive Spearman-ρ Karte: BIP vs. Militär'
)

fig.update_geos(showcoastlines=True, showland=True, fitbounds='locations')
fig.update_layout(margin={'r':0,'t':40,'l':0,'b':0})
fig.show()

# Korrelation Importe - Militärausgaben

In [47]:
df_military = pd.read_csv("data/military_expenditure.csv", sep=',', skiprows=4)
df_military = df_military.drop(columns=["Indicator Name", "Indicator Code", "2024", "Unnamed: 69"])
df_military = clean_df(df_military)

In [48]:
df_import = pd.read_csv("data/imports.csv", sep=',', skiprows=4)
df_import = df_import.drop(columns=["Indicator Name", "Indicator Code", "2024", "Unnamed: 69"])
df_import = clean_df(df_import)

In [49]:
df_import, df_military = filter_for_same_columns(df_import, df_military)


In [50]:
df = df_import.merge(df_military, on="Country Name", suffixes=("_imp", "_mil"))

years = [c for c in df_import.columns if c.isdigit() and c in df_military.columns]

results = []
for _, row in df.iterrows():
    country = row["Country Name"]
    # Zeitreihen in numpy-Arrays
    x = pd.to_numeric(row[[f"{y}_imp" for y in years]], errors="coerce").to_numpy()
    y = pd.to_numeric(row[[f"{y}_mil" for y in years]], errors="coerce").to_numpy()
    
    # Nur valide Paare (nicht-NaN, nicht 0)
    mask = (~np.isnan(x)) & (x != 0) & (~np.isnan(y)) & (y != 0)
    x_valid = x[mask]
    y_valid = y[mask]
    
    if len(x_valid) < 3:
        # für mindestens 3 Wertepaare sinnvoll
        continue
    
    # Spearman ρ
    rho, p_s = spearmanr(x_valid, y_valid)
    # Kendall's τ_b
    tau, p_k = kendalltau(x_valid, y_valid)
    
    results.append({
        "Country Name": country,
        "n Jahre": len(x_valid),
        "Spearman ρ": round(rho, 3),
        "p-Spearman": round(p_s, 3),
        "Kendall τ_b": round(tau, 3),
        "p-Kendall": round(p_k, 3)
    })

# 5) In DataFrame und CSV
df_country_corr = pd.DataFrame(results) \
    .sort_values("Spearman ρ", ascending=False) \
    .reset_index(drop=True)

print(df_country_corr)
df_country_corr.to_csv("results/import_correlations.csv", index=False)

               Country Name  n Jahre  Spearman ρ  p-Spearman  Kendall τ_b  \
0               Afghanistan        8       1.000       0.000        1.000   
1                     Nepal       54       0.992       0.000        0.937   
2                     China       35       0.992       0.000        0.946   
3                 Singapore       54       0.991       0.000        0.943   
4                Luxembourg       54       0.990       0.000        0.939   
..                      ...      ...         ...         ...          ...   
153                  Guyana       30      -0.117       0.539       -0.053   
154  Bosnia and Herzegovina       22      -0.240       0.282       -0.169   
155            Turkmenistan        6      -0.314       0.544       -0.333   
156                 Lao PDR       22      -0.319       0.148       -0.134   
157             South Sudan        8      -0.595       0.120       -0.286   

     p-Kendall  
0        0.000  
1        0.000  
2        0.000  
3      

In [51]:
df_corr = pd.read_csv('results/import_correlations.csv')

def iso3(name):
    try:
        return pycountry.countries.lookup(name).alpha_3
    except LookupError:
        return None

df_corr['iso_a3'] = df_corr['Country Name'].apply(iso3)

df_map = df_corr.dropna(subset=['iso_a3'])

fig = px.choropleth(
    df_map,
    locations='iso_a3',
    color='Spearman ρ',
    hover_name='Country Name',
    color_continuous_scale='OrRd',
    projection='natural earth',
    title='Interaktive Spearman-ρ Karte: Importe vs. Militär'
)

fig.update_geos(showcoastlines=True, showland=True, fitbounds='locations')
fig.update_layout(margin={'r':0,'t':40,'l':0,'b':0})
fig.show()

# Korrelation Exportüberschuss - Militärausgaben

In [52]:
df_military = pd.read_csv("data/military_expenditure.csv", sep=',', skiprows=4)
df_military = df_military.drop(columns=["Indicator Name", "Indicator Code", "2024", "Unnamed: 69"])
df_military = clean_df(df_military)

In [53]:
df_import = pd.read_csv("data/imports.csv", sep=',', skiprows=4)
df_import = df_import.drop(columns=["Indicator Name", "Indicator Code", "2024", "Unnamed: 69"])
df_import = clean_df(df_import)

In [54]:
df_export = pd.read_csv("data/exports.csv", sep=',', skiprows=4)
df_export = df_export.drop(columns=["Indicator Name", "Indicator Code", "2024", "Unnamed: 69"])
df_export = clean_df(df_export)

In [55]:
year_cols = [c for c in df_export.columns if c.isdigit()]

# 3) Setze Country Name als Index (optional, macht's übersichtlicher)
df_exp2 = df_export.set_index("Country Name")
df_imp2 = df_import.set_index("Country Name")

# 4) Berechne den Überschuss je Jahr
df_surplus = df_exp2[year_cols] - df_imp2[year_cols]

# 5) (Optional) Index zurück als Spalte
df_surplus = df_surplus.reset_index()

# 6) Ergebnis anschauen und speichern
print(df_surplus.head())

          Country Name          1960          1961          1962  \
0          Afghanistan -1.555555e+07 -1.999999e+07 -2.444443e+07   
1              Albania           NaN           NaN           NaN   
2              Algeria -7.653484e+08 -5.176081e+08 -2.051028e+07   
3               Angola           NaN           NaN           NaN   
4  Antigua and Barbuda           NaN           NaN           NaN   

           1963          1964          1965          1966          1967  \
0 -5.777780e+07 -7.333331e+07 -1.022222e+08 -1.400000e+08 -1.244444e+08   
1           NaN           NaN           NaN           NaN           NaN   
2 -3.281645e+08 -1.267212e+08 -1.012748e+08  4.050994e+07  6.076491e+07   
3           NaN           NaN           NaN           NaN           NaN   
4           NaN           NaN           NaN           NaN           NaN   

           1968  ...          2014          2015          2016          2017  \
0 -8.666667e+07  ...           NaN           NaN           N

In [56]:
df_surplus, df_military = filter_for_same_columns(df_surplus, df_military)

In [57]:
df = df_surplus.merge(df_military, on="Country Name", suffixes=("_surp", "_mil"))

years = [c for c in df_surplus.columns if c.isdigit() and c in df_military.columns]

results = []
for _, row in df.iterrows():
    country = row["Country Name"]

    x = pd.to_numeric(row[[f"{y}_surp" for y in years]], errors="coerce").to_numpy()
    y = pd.to_numeric(row[[f"{y}_mil" for y in years]], errors="coerce").to_numpy()
    
    mask = (~np.isnan(x)) & (x != 0) & (~np.isnan(y)) & (y != 0)
    x_valid = x[mask]
    y_valid = y[mask]
    
    if len(x_valid) < 3:
        continue
    
    # Spearman ρ
    rho, p_s = spearmanr(x_valid, y_valid)
    # Kendall's τ_b
    tau, p_k = kendalltau(x_valid, y_valid)
    
    results.append({
        "Country Name": country,
        "n Jahre": len(x_valid),
        "Spearman ρ": round(rho, 3),
        "p-Spearman": round(p_s, 3),
        "Kendall τ_b": round(tau, 3),
        "p-Kendall": round(p_k, 3)
    })

# 5) In DataFrame und CSV
df_country_corr = pd.DataFrame(results) \
    .sort_values("Spearman ρ", ascending=False) \
    .reset_index(drop=True)

print(df_country_corr)
df_country_corr.to_csv("results/surplus_correlations.csv", index=False)

        Country Name  n Jahre  Spearman ρ  p-Spearman  Kendall τ_b  p-Kendall
0          Singapore       54       0.980         0.0        0.898        0.0
1         Luxembourg       54       0.961         0.0        0.842        0.0
2        Netherlands       55       0.943         0.0        0.797        0.0
3              Qatar       11       0.936         0.0        0.855        0.0
4            Denmark       58       0.910         0.0        0.722        0.0
..               ...      ...         ...         ...          ...        ...
152  Kyrgyz Republic       30      -0.916         0.0       -0.766        0.0
153         Honduras       57      -0.916         0.0       -0.760        0.0
154        Sri Lanka       58      -0.941         0.0       -0.792        0.0
155       Bangladesh       51      -0.955         0.0       -0.835        0.0
156            Nepal       54      -0.986         0.0       -0.913        0.0

[157 rows x 6 columns]


In [58]:
df_corr = pd.read_csv('results/surplus_correlations.csv')

def iso3(name):
    try:
        return pycountry.countries.lookup(name).alpha_3
    except LookupError:
        return None

df_corr['iso_a3'] = df_corr['Country Name'].apply(iso3)

df_map = df_corr.dropna(subset=['iso_a3'])

fig = px.choropleth(
    df_map,
    locations='iso_a3',
    color='Spearman ρ',
    hover_name='Country Name',
    color_continuous_scale='OrRd',
    projection='natural earth',
    title='Interaktive Spearman-ρ Karte: Exportüberschuss vs. Militär'
)

fig.update_geos(showcoastlines=True, showland=True, fitbounds='locations')
fig.update_layout(margin={'r':0,'t':40,'l':0,'b':0})
fig.show()